In [ ]:
import io
import base64
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
import cv2  # OpenCV for video processing
from skimage.transform import resize
from skimage.color import rgb2gray
from sklearn.linear_model import RidgeClassifier
from scipy.fftpack import dct
with open('models/frequencymodel.pkl', 'rb') as f:
    ridge_clf = pickle.load(f)


def extract_dct(img):
    img = dct(img, type=2, norm="ortho", axis=0)
    img = dct(img, type=2, norm="ortho", axis=1)
    img = np.abs(img)
    img += 1e-13
    img = np.log(img)
    img -= np.mean(img)
    img /= np.std(img)
    
    # Return both flattened features and DCT image
    return img.flatten(), img

def predict_video(video_path,image_size=(128, 128)):
    with open('models/frequencymodel.pkl', 'rb') as f:
        ridge_clf = pickle.load(f)
    cap = cv2.VideoCapture(video_path)
    predictions = []
    dct_images = []
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = resize(frame, image_size, anti_aliasing=True)
        frame_gray = rgb2gray(frame_resized)

        dct_features, dct_image = extract_dct(frame_gray)
        prediction = model.predict([dct_features])[0]
        probability = model.decision_function([dct_features])[0]

        predictions.append((prediction, probability))
        dct_images.append(dct_image)
        frames.append(frame)

    cap.release()

    highest_prob_index = np.argmax([prob for _, prob in predictions])
    most_probable_frame = frames[highest_prob_index]
    most_probable_dct_image = dct_images[highest_prob_index]

    # Create an in-memory byte buffer for the frame and DCT plot
    frame_buffer = io.BytesIO()
    dct_buffer = io.BytesIO()

    # Save the frame to the buffer
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(most_probable_frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.savefig(frame_buffer, format='png')
    frame_buffer.seek(0)  # Move the pointer to the start of the buffer

    # Save the DCT plot to the buffer
    plt.figure(figsize=(6, 6))
    plt.imshow(most_probable_dct_image, cmap='inferno')
    plt.colorbar()
    plt.savefig(dct_buffer, format='png')
    dct_buffer.seek(0)

    # Encode the buffers to base64 for sending over HTTP
    frame_base64 = base64.b64encode(frame_buffer.read()).decode('utf-8')
    dct_base64 = base64.b64encode(dct_buffer.read()).decode('utf-8')

    # Return the prediction result and the base64 images
    return {
        "prediction": int(np.round(np.mean([pred for pred, _ in predictions]))),
        "frame_base64": frame_base64,
        "dct_base64": dct_base64
    }
